In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate simulated data for RIS elements' characteristics (channel conditions, energy levels, signal reception)
num_elements = 300

elements = np.random.normal(0.5, 0.1, (num_elements, 4))

# Define channel gain and Rayleigh fading parameters
path_loss_exponent = 2.0
noise_power = 0.01

# Genetic Algorithm for optimal redistribution using random initialization
population_size = 100
num_generations = 100
crossover_rate = 0.8
mutation_rate = 0.1

best_snr = float('inf')
best_power = float('inf')
best_energy_indices = None
best_signal_indices = None

def calculate_snr_power(energy_indices, signal_indices):
    # Define the fading channel gain
    h = 0.5 # You can update this value based on your specific requirements
    # Define the power of the base station
    P_bs = 1 # The power of the base station is 1 W
    # Calculate the energy and signal power
    energy_power = np.sum(elements[energy_indices, 1])
    signal_power = np.sum(elements[signal_indices, 1])
    # Calculate the SNR
    snr = (P_bs * h * signal_power) / (P_bs * h * energy_power + noise_power)
    # Calculate the total power
    power = energy_power + signal_power
    return snr, power




def evaluate_fitness(energy_indices, signal_indices):
    snr, power = calculate_snr_power(energy_indices, signal_indices)
    return 1 / (snr * power)  # Minimize the reciprocal of SNR * power

def genetic_algorithm():
    population = [np.random.choice(range(num_elements), num_elements // 2, replace=False) for _ in range(population_size)]
    
    for _ in range(num_generations):
        new_population = []
        for parent1, parent2 in zip(population[::2], population[1::2]):
            if np.random.rand() < crossover_rate:
                crossover_point = np.random.randint(1, num_elements // 2)
                child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
                child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            else:
                child1 = parent1
                child2 = parent2
            
            for child in [child1, child2]:
                if np.random.rand() < mutation_rate:
                    mutation_index = np.random.randint(num_elements // 2)  # Ensure the mutated index is within bounds
                    child[mutation_index] = np.random.randint(num_elements)        
            new_population.extend([child1, child2])
        
        population = new_population
        population.sort(key=lambda ind: evaluate_fitness(ind, list(set(range(num_elements)) - set(ind))))
        # population = population[:population_size]

        
        best_individual = population[0]
        best_energy_indices = best_individual
        best_signal_indices = list(set(range(num_elements)) - set(best_individual))
        best_snr, best_power = calculate_snr_power(best_energy_indices, best_signal_indices)
        
    return best_energy_indices, best_signal_indices, best_snr, best_power

def smarter_optimization(energy_indices, signal_indices):
    initial_snr, initial_power = calculate_snr_power(energy_indices, signal_indices)
    best_energy_indices = energy_indices.copy()
    best_signal_indices = signal_indices.copy()
    best_combined_improvement = 0
    
    for _ in range(100):  # Perform 10 iterations
        improved = False
        for energy_index in energy_indices:
            for signal_index in signal_indices:
                new_energy_indices = [idx for idx in energy_indices if idx != energy_index]
                new_signal_indices = [idx for idx in signal_indices if idx != signal_index]
                new_energy_indices.append(signal_index)
                new_signal_indices.append(energy_index)
                
                new_snr, new_power = calculate_snr_power(new_energy_indices, new_signal_indices)
                snr_improvement = new_snr - initial_snr
                power_reduction = initial_power - new_power
                combined_improvement_metric = snr_improvement - 0.5 * power_reduction
                
                if combined_improvement_metric > best_combined_improvement:
                    best_combined_improvement = combined_improvement_metric
                    best_energy_indices = new_energy_indices
                    best_signal_indices = new_signal_indices
                    improved = True
        
        if not improved:
            break
    
    return best_energy_indices, best_signal_indices, best_combined_improvement

# Run the Genetic Algorithm
best_energy_indices, best_signal_indices, best_snr, best_power = genetic_algorithm()

# Calculate initial SNR and power
initial_snr, initial_power = calculate_snr_power(range(num_elements // 2), range(num_elements // 2, num_elements))


# Apply the smarter optimization strategy
best_energy_indices, best_signal_indices, combined_improvement_metric = smarter_optimization(best_energy_indices, best_signal_indices)
# Calculate efficiency metrics
snr_improvement_percentage = (abs(best_snr - initial_snr) / initial_snr) * 100
power_reduction_percentage = (abs(initial_power - best_power) / initial_power) * 100
energy_efficiency = (best_snr * best_power) / num_elements
combined_improvement_metric = ((snr_improvement_percentage * power_reduction_percentage) ** 0.5)

# Print optimal groupings and their characteristics
print("Optimal Energy Group Indices:", best_energy_indices)
print("Optimal Signal Group Indices:", best_signal_indices)
print("Initial SNR:", initial_snr, "dB")
print("Initial Power:", initial_power, "mW")
print("Optimal SNR:", best_snr, "dB")
print("Optimal Power:", best_power, "mW")
print("SNR Improvement Percentage:", snr_improvement_percentage, "%")
print("Power Reduction Percentage:", power_reduction_percentage, "%")
print("Energy Efficiency:", energy_efficiency, "SNR * Power / Num Elements")
print("Combined Improvement Metric:", combined_improvement_metric)

# Visualize the efficiency
plt.figure(figsize=(8, 6))
plt.scatter(elements[best_energy_indices, 0] / elements[best_energy_indices, 2], elements[best_energy_indices, 1], label='Energy Group')
plt.scatter(elements[best_signal_indices, 0] / elements[best_signal_indices, 2], elements[best_signal_indices, 1], label='Signal Group')
plt.xlabel('SNR (dB)')
plt.ylabel('Power (mW)')
plt.title('Optimal RIS Element Redistribution (Genetic Algorithm)')
plt.legend()
plt.grid(True)
plt.show()



+ smart optimization

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Number of RIS elements
num_elements = 300

# Define positions
user_position = (100, 100)
base_station_position = (0, 0)
uav_position = (50, 50)  # Example UAV-RIS position

# Genetic Algorithm parameters
population_size = 100
num_generations = 100
crossover_rate = 0.8
mutation_rate = 0.1
noise_power = 0.01

# SNR with UAV-RIS (Unoptimized)
snr_uav_unoptimized = 0


def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)


user_bs_distance = calculate_distance(user_position, base_station_position)
user_uav_distance = calculate_distance(user_position, uav_position)
bs_uav_distance = calculate_distance(base_station_position, uav_position)

# Generate channel coefficients and fading
channel_coefficients = np.random.rand(num_elements)
fading = np.random.rand(num_elements)
# rayleigh_fading = np.random.rayleigh(scale=1, size=num_elements)


def calculate_elements(channel_coefficients, fading):
    snr_values = (
        channel_coefficients
        * fading
        / (channel_coefficients * (1 - fading) + noise_power)
    )
    power_values = channel_coefficients * fading + channel_coefficients * (1 - fading)
    return np.column_stack((snr_values, power_values))


elements = calculate_elements(channel_coefficients, fading)


def calculate_snr_power(channel_coefficients, fading):
    signal_power = np.sum(channel_coefficients * fading)
    energy_power = np.sum(channel_coefficients * (1 - fading))
    snr = signal_power / (energy_power + noise_power)
    power = energy_power + signal_power
    return snr, power


def generate_initial_indices(channel_coefficients, fading):
    sorted_indices = np.argsort(channel_coefficients * fading)
    energy_indices = sorted_indices[: num_elements // 2]
    signal_indices = sorted_indices[num_elements // 2 :]
    return energy_indices, signal_indices


def evaluate_fitness(energy_indices, signal_indices):
    snr, power = calculate_snr_power(
        channel_coefficients[energy_indices], fading[energy_indices]
    )
    return 1 / (snr * power)


def genetic_algorithm():
    # Generate initial values for energy_indices and signal_indices
    energy_indices, signal_indices = generate_initial_indices(
        channel_coefficients, fading
    )

    # Initialize population
    population = [energy_indices] * population_size

    for i in range(num_generations):
        new_population = []
        for parent1, parent2 in zip(population[::2], population[1::2]):
            if np.random.rand() < crossover_rate:
                crossover_point = np.random.randint(1, num_elements // 2)
                child1 = np.concatenate(
                    (parent1[:crossover_point], parent2[crossover_point:])
                )
                child2 = np.concatenate(
                    (parent2[:crossover_point], parent1[crossover_point:])
                )
            else:
                child1 = parent1
                child2 = parent2

            for child in [child1, child2]:
                if np.random.rand() < mutation_rate:
                    mutation_index = np.random.randint(num_elements // 2)
                    child[mutation_index] = np.random.randint(num_elements)
            new_population.extend([child1, child2])

        population = new_population

        best_individual = min(
            population,
            key=lambda ind: evaluate_fitness(
                ind, list(set(range(num_elements)) - set(ind))
            ),
        )

        best_energy_indices = best_individual
        best_signal_indices = list(set(range(num_elements)) - set(best_individual))
        best_snr, best_power = calculate_snr_power(
            channel_coefficients[best_energy_indices], fading[best_energy_indices]
        )

        if i == 0:
            global snr_uav_unoptimized
            snr_uav_unoptimized = best_snr

    return best_energy_indices, best_signal_indices, best_snr, best_power


# Run the Genetic Algorithm
best_energy_indices, best_signal_indices, best_snr, best_power = genetic_algorithm()

# Desired initial SNR without UAV-RIS in dB
desired_initial_snr_dB = 0.2

# Convert desired initial SNR from dB to linear scale
desired_initial_snr = 10 ** (desired_initial_snr_dB / 10)

# Calculate the required base station power to achieve the desired initial SNR
P_bs = (user_bs_distance**path_loss_exponent + noise_power) * desired_initial_snr

# SNR directly from the base station (Without UAV-RIS)
snr_without_ris = P_bs / (user_bs_distance**path_loss_exponent + noise_power)

# Calculate the improvement percentage
improvement_percentage = (
    (best_snr - 10 * np.log10(snr_without_ris)) / snr_without_ris * 100
)


# Print optimal groupings and their characteristics
print("Optimal Energy Group Indices:", best_energy_indices)
print("Optimal Signal Group Indices:", best_signal_indices)
print("SNR Without UAV-RIS:", 10 * np.log10(snr_without_ris), "dB")
print("SNR With UAV-RIS (Unoptimized):", snr_uav_unoptimized, "dB")
print("SNR With UAV-RIS (Optimized):", best_snr, "dB")
print("Optimal Power (Optimized):", best_power, "mW")
print("Improvement Percentage:", improvement_percentage, "%")


# Visualize
plt.figure(figsize=(6, 6))
plt.scatter(
    channel_coefficients[best_energy_indices],
    fading[best_energy_indices],
    label="Energy Group",
)
plt.scatter(
    channel_coefficients[best_signal_indices],
    fading[best_signal_indices],
    label="Signal Group",
)
plt.xlabel("Channel Coefficients")
plt.ylabel("Fading")
plt.title("Optimal RIS Element Redistribution (Genetic Algorithm)")
plt.legend()
plt.grid(True)
plt.show()

# Visualize the distribution based on SNR and Power
plt.figure(figsize=(6, 6))
plt.scatter(
    elements[best_energy_indices, 0],
    elements[best_energy_indices, 1],
    label="Energy Group",
)
plt.scatter(
    elements[best_signal_indices, 0],
    elements[best_signal_indices, 1],
    label="Signal Group",
)
plt.xlabel("SNR")
# x from 0 to 10
plt.xlim(-1, 10)
plt.ylabel("Power (mW)")
plt.title("Optimal RIS Element Redistribution (Genetic Algorithm)")
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'path_loss_exponent' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from heapq import heappop, heappush

# Define the grid environment
grid_size = 11
num_obstacles = 80
obstacles = [(np.random.randint(0, grid_size), np.random.randint(0, grid_size)) for _ in range(num_obstacles)]
start = (0, 0)
goal = (grid_size - 1, grid_size - 1)

# A* pathfinding with diagonal movements
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(start, goal, max_iterations=30000):
    open_list = [(0, start)]
    came_from = {}
    g_score = {start: 0}
    iterations = 0

    while open_list and iterations < max_iterations:
        current_g, current = heappop(open_list)
        if current == goal:
            path = [goal]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            path.reverse()
            return path

        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < grid_size and 0 <= neighbor[1] < grid_size and neighbor not in obstacles:
                tentative_g = g_score[current] + np.sqrt(dx ** 2 + dy ** 2)
                if tentative_g < g_score.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score = tentative_g + heuristic(neighbor, goal)
                    heappush(open_list, (f_score, neighbor))

        iterations += 1

    return None

def d_star_lite(start, goal):
    open_list = [(0, goal)]
    came_from = {}
    g_score = {goal: 0}

    while open_list:
        current_g, current = heappop(open_list)
        if current == start:
            path = [start]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            path.reverse()
            return path

        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < grid_size and 0 <= neighbor[1] < grid_size and neighbor not in obstacles:
                tentative_g = g_score[current] + np.sqrt(dx ** 2 + dy ** 2)
                if tentative_g < g_score.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score = tentative_g + heuristic(neighbor, start)
                    heappush(open_list, (f_score, neighbor))

    return None

def bfs(start, goal):
    queue = [start]
    came_from = {}
    visited = set()

    while queue:
        current = queue.pop(0)
        visited.add(current)

        if current == goal:
            path = [goal]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            path.reverse()
            return path

        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < grid_size and 0 <= neighbor[1] < grid_size and neighbor not in obstacles and neighbor not in visited:
                queue.append(neighbor)
                came_from[neighbor] = current

    return None

def dijkstra(start, goal):
    open_list = [(0, start)]
    came_from = {}
    g_score = {start: 0}

    while open_list:
        current_g, current = heappop(open_list)
        if current == goal:
            path = [goal]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            path.reverse()
            return path

        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < grid_size and 0 <= neighbor[1] < grid_size and neighbor not in obstacles:
                tentative_g = g_score[current] + np.sqrt(dx ** 2 + dy ** 2)
                if tentative_g < g_score.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score = tentative_g
                    heappush(open_list, (f_score, neighbor))

    return None

# Find optimal path using A* pathfinding
path_astar = astar(start, goal, max_iterations=50000)

# Find optimal path using D* Lite
path_d_star_lite = d_star_lite(start, goal)

# Find optimal path using Breadth-First Search
path_bfs = bfs(start, goal)

# Find optimal path using Dijkstra's algorithm
path_dijkstra = dijkstra(start, goal)

# Calculate path lengths
path_lengths = {
    "A*": len(path_astar) if path_astar is not None else float('inf'),
    "D* Lite": len(path_d_star_lite) if path_d_star_lite is not None else float('inf'),
    "BFS": len(path_bfs) if path_bfs is not None else float('inf'),
    "Dijkstra's": len(path_dijkstra) if path_dijkstra is not None else float('inf')
}

# Store obstacle coordinates and optimal path coordinates
# optimal_path_coords = best_option  # Replace with the variable storing the optimal path coordinates
stored_obstacles = obstacles.copy()

# ... (rest of the code)

# Use the stored obstacle coordinates and optimal path coordinates in the third and fourth snippets
# Pass stored_obstacles and optimal_path_coords as inputs to the relevant functions

# Find the shortest path
best_option = min(path_lengths, key=path_lengths.get)

# Print the best option
print("Best Path Option:", best_option)

# Visualize the paths and obstacles
plt.figure(figsize=(10, 10))

plt.scatter([o[0] for o in obstacles], [o[1] for o in obstacles], color='red', marker='x', label='Obstacles')

# Plot all paths
if path_astar is not None:
    plt.plot([p[0] for p in path_astar], [p[1] for p in path_astar], marker='o', linestyle='-', color='blue', label='A* Path')

if path_d_star_lite is not None:
    plt.plot([p[0] for p in path_d_star_lite], [p[1] for p in path_d_star_lite], marker='o', linestyle='-', color='green', label='D* Lite Path')

if path_bfs is not None:
    plt.plot([p[0] for p in path_bfs], [p[1] for p in path_bfs], marker='o', linestyle='-', color='orange', label='BFS Path')

if path_dijkstra is not None:
    plt.plot([p[0] for p in path_dijkstra], [p[1] for p in path_dijkstra], marker='o', linestyle='-', color='red', label="Dijkstra's Path")

plt.scatter(start[0], start[1], color='green', marker='o', label='Start')
plt.scatter(goal[0], goal[1], color='purple', marker='o', label='Goal')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Pathfinding Algorithm Comparison')
plt.legend()
plt.grid(True)
plt.show()

working RIS opti(genetic algo) with path finding

previous version
+++

Smarter Optimization Strategy: Instead of random removal and addition of RIS elements, you could implement a more sophisticated optimization strategy. For example, you could use gradient-based optimization techniques or heuristics to intelligently select RIS elements for removal and addition based on their impact on SNR improvement, power reduction, and energy efficiency.

Dynamic Optimization Parameters: Adjust the optimization parameters dynamically based on the point's characteristics or the path's progression. For instance, you could increase the optimization factor or mutation rate when the path is closer to obstacles or the goal to encourage more significant changes in RIS distribution.

Multi-Objective Optimization: Optimize for multiple objectives simultaneously, such as maximizing SNR improvement while minimizing power reduction. This can be achieved by assigning different weights to SNR improvement and power reduction, and then optimizing for a combination of these objectives.

Constraint Handling: Apply constraints on the optimization process to ensure that SNR improvement and power reduction are both positive. For example, you can prevent removing too many RIS elements, which could lead to power reduction dominating the efficiency metrics.

Iterative Refinement: Allow for multiple iterations of RIS optimization at each point along the path, gradually refining the RIS distribution to improve efficiency metrics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from heapq import heappop, heappush

# Define the grid environment
grid_size = 11
num_obstacles = 50
obstacles = [(np.random.randint(0, grid_size), np.random.randint(0, grid_size)) for _ in range(num_obstacles)]
start = (0, 0)
goal = (grid_size - 1, grid_size - 1)

# Generate simulated data for RIS elements' characteristics (channel conditions, energy levels, signal reception)
num_elements = 100
elements = np.random.normal(0.5, 0.1, (num_elements, 4))


# Genetic Algorithm for optimal redistribution of RIS elements
def genetic_algorithm(population_size=100, num_generations=100, mutation_rate=0.1):
    population = [np.random.choice(range(num_elements), num_elements // 2, replace=False) for _ in range(population_size)]
    
    for _ in range(num_generations):
        new_population = []
        
        for _ in range(population_size // 2):
            parent1_idx = np.random.randint(population_size)
            parent2_idx = np.random.randint(population_size)
            parent1 = population[parent1_idx]
            parent2 = population[parent2_idx]
            
            crossover_point = np.random.randint(1, num_elements // 2)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            
            if np.random.rand() < mutation_rate:
                mutation_index = np.random.randint(num_elements // 2)
                child1[mutation_index] = np.random.randint(num_elements)
            
            if np.random.rand() < mutation_rate:
                mutation_index = np.random.randint(num_elements // 2)
                child2[mutation_index] = np.random.randint(num_elements)
            
            new_population.extend([child1, child2])
        
        population = new_population
    
    # Calculate SNR and power for each candidate solution
    candidate_metrics = [calculate_snr_power(candidate, list(set(range(num_elements)) - set(candidate))) for candidate in population]
    
    # Choose the solution with the best SNR
    best_candidate_idx = np.argmax([metrics[0] for metrics in candidate_metrics])
    best_energy_indices = population[best_candidate_idx]
    best_signal_indices = list(set(range(num_elements)) - set(best_energy_indices))
    best_snr, best_power = candidate_metrics[best_candidate_idx]
    
    return best_energy_indices, best_signal_indices, best_snr, best_power


# A* pathfinding with diagonal movements and RIS optimization
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# Calculate SNR and power based on element indices
def calculate_snr_power(energy_indices, signal_indices):
    energy_power = np.sum(elements[energy_indices, 1])  # Total energy power
    signal_power = np.sum(elements[signal_indices, 1])  # Total signal power

    energy_distance = np.sum(elements[energy_indices, 2])  # Sum of distances for energy elements
    signal_distance = np.sum(elements[signal_indices, 2])  # Sum of distances for signal elements

    # Add epsilon to avoid division by zero and negative values
    epsilon = 1e-10

    # Calculate SNR considering Friis free space model
    if energy_power < epsilon or energy_distance < epsilon:
        snr = 0
    else:
        snr = (signal_power / (signal_distance**2 + epsilon)) / (energy_power / (energy_distance**2 + epsilon))

    # Introduce Rayleigh fading effect
    snr *= np.random.rayleigh()

    power = energy_power + signal_power  # Total power
    return snr, power

def astar_with_ris(start, goal, max_iterations=30000):
    open_list = [(0, start)]
    came_from = {}
    g_score = {start: 0}
    iterations = 0

    while open_list and iterations < max_iterations:
        current_g, current = heappop(open_list)
        if current == goal:
            path = [goal]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            path.reverse()
            return path

        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < grid_size and 0 <= neighbor[1] < grid_size and neighbor not in stored_obstacles:  # Use the stored obstacles
                tentative_g = g_score[current] + np.sqrt(dx ** 2 + dy ** 2)
                if tentative_g < g_score.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    snr_improvement, power_reduction = calculate_snr_power(best_energy_indices, best_signal_indices)  # Use best_energy_indices and best_signal_indices here
                    pathfinding_cost = heuristic(neighbor, goal)
                    w_pathfinding = 0.25  # Weight for pathfinding cost
                    w_snr_improvement = 0.4  # Weight for SNR improvement
                    w_power_reduction = 0.35  # Weight for power reduction
                    total_cost = w_pathfinding * pathfinding_cost + w_snr_improvement * snr_improvement + w_power_reduction * power_reduction
                    heappush(open_list, (total_cost, neighbor))

        iterations += 1

    return None

# Define the weights for optimization factors
w_snr_improvement = 0.6
w_power_reduction = 0.4

# Optimize RIS distribution at a point
def optimize_ris_distribution_at_point(point, current_energy_indices, current_signal_indices, iteration, max_iterations):
    # Calculate the distance from the current point to the goal
    distance_to_goal = np.sqrt((goal[0] - point[0]) ** 2 + (goal[1] - point[1]) ** 2)
    
    # Determine the optimization factor based on distance to goal
    optimization_factor = 1.0 - distance_to_goal / np.sqrt(grid_size ** 2 + grid_size ** 2)
    
    # Define epsilon
    epsilon = 1e-10
    
    # Initialize arrays to keep track of indices to remove and add
    indices_to_remove = []
    indices_to_add = []
    
    # Define weights for SNR improvement and power reduction
    w_snr_improvement = 0.5
    w_power_reduction = 0.5
    
    # Adjust weights dynamically based on conditions
    if iteration < max_iterations / 4:
        w_snr_improvement = 0.6
        w_power_reduction = 0.4
    # You can add more conditions to adjust weights as needed
    
    for idx in range(len(current_energy_indices)):
        if np.random.rand() < optimization_factor:
            indices_to_remove.append(idx)
            indices_to_add.append(idx)
    
    # Remove indices from updated_energy_indices
    updated_energy_indices = np.delete(current_energy_indices, indices_to_remove)
    
    # Add indices to updated_signal_indices
    updated_signal_indices = np.append(current_signal_indices, current_energy_indices[indices_to_add])
     
    # Calculate SNR and power based on the updated distribution
    new_snr, new_power = calculate_snr_power(updated_energy_indices, updated_signal_indices)
    old_snr, old_power = calculate_snr_power(current_energy_indices, current_signal_indices)
    
    # Handle cases where division by zero or near-zero occurs
    snr_improvement = new_snr - old_snr
    power_reduction = old_power - new_power
    
    # Apply constraints to ensure non-negative efficiency metrics
    if snr_improvement < epsilon:
        snr_improvement = epsilon
    if power_reduction < epsilon:
        power_reduction = epsilon
    
    # Calculate energy efficiency using weighted combination
    energy_efficiency = (w_snr_improvement * snr_improvement) / (w_power_reduction * power_reduction)
    
    # Calculate the combined improvement metric
    combined_improvement_metric = w_snr_improvement * snr_improvement + w_power_reduction * power_reduction
    
    return updated_energy_indices, updated_signal_indices, snr_improvement, power_reduction, energy_efficiency, combined_improvement_metric

max_iterations = 50000
# Run the Genetic Algorithm
best_energy_indices, best_signal_indices, best_snr, best_power = genetic_algorithm()

# Find optimal path using A* pathfinding with RIS optimization
path_with_ris = astar_with_ris(start, goal, max_iterations=max_iterations)

if path_with_ris is not None:
    # Optimize RIS element distribution at every point along the path and calculate efficiencies
    optimized_efficiency_metrics = []
    current_energy_indices = best_energy_indices.copy()
    current_signal_indices = best_signal_indices.copy()
    
    for i, point in enumerate(path_with_ris):
        current_energy_indices, current_signal_indices, snr_improvement, power_reduction, energy_efficiency, combined_improvement_metric = optimize_ris_distribution_at_point(point, current_energy_indices, current_signal_indices, i, max_iterations)
        optimized_efficiency_metrics.append((i, point, snr_improvement, power_reduction, energy_efficiency, combined_improvement_metric))
   
    # Visualize the efficiency metrics along the path
    if optimized_efficiency_metrics:
        plt.figure(figsize=(8, 6))
        plt.plot([i for i, _, _, _, _, _ in optimized_efficiency_metrics], [combined_improvement_metric for _, _, _, _, _, combined_improvement_metric in optimized_efficiency_metrics], marker='o', linestyle='-', color='blue', label='Combined Improvement Metric')
        plt.xlabel('Point along Path')
        plt.ylabel('Combined Improvement Metric')
        plt.title('Efficiency Metrics along Optimal Path with RIS Optimization')
        plt.legend()
        plt.grid(True)
        plt.show()
    
    # Visualize the original and optimized paths along with obstacles
    plt.figure(figsize=(10, 10))
    plt.plot([p[0] for p in path_with_ris], [p[1] for p in path_with_ris], marker='o', linestyle='-', color='blue', label='Optimized Path')
    plt.scatter([o[0] for o in obstacles], [o[1] for o in obstacles], color='red', marker='x', label='Obstacles')
    plt.scatter(start[0], start[1], color='green', marker='o', label='Start')
    plt.scatter(goal[0], goal[1], color='purple', marker='o', label='Goal')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Optimal Path Planning with RIS Optimization using Genetic Algorithm')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("Path not found.")

